In [1]:
# Licensing Information:  You are free to use or extend this project for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to The Georgia Institute of Technology, including a link to https://aritter.github.io/CS-7650/

# Attribution Information: This assignment was developed at The Georgia Institute of Technology
# by Alan Ritter (alan.ritter@cc.gatech.edu)

# Project #2: Named Entity Recognition

In this assignment, you will implement a bidirectional LSTM-CNN-CRF for sequence labeling, following [this paper by Xuezhe Ma and Ed Hovy](https://www.aclweb.org/anthology/P16-1101.pdf), on the CoNLL named entity recognition dataset.  Before starting the assignment, we recommend reading the Ma and Hovy paper.

First, let's import some libraries and make sure the runtime has access to a GPU.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Sat Mar 13 01:36:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download the Data

Run the following code to download the English part of the CoNLL 2003 dataset, the evaluation script and pre-filtered GloVe embeddings we are providing for this data.

In [3]:
#CoNLL 2003 data
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testb
!cat eng.train | awk '{print $1 "\t" $4}' > train
!cat eng.testa | awk '{print $1 "\t" $4}' > dev
!cat eng.testb | awk '{print $1 "\t" $4}' > test

#Evaluation Script
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl

#Pre-filtered GloVe embeddings
!wget https://raw.githubusercontent.com/aritter/aritter.github.io/master/files/glove.840B.300d.conll_filtered.txt

--2021-03-13 01:36:02--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283420 (3.1M) [text/plain]
Saving to: ‘eng.train’

eng.train           100%[===================>]   3.13M  --.-KB/s    in 0.03s   

2021-03-13 01:36:02 (109 MB/s) - ‘eng.train’ saved [3283420/3283420]

--2021-03-13 01:36:02--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response..

## CoNLL Data Format

Run the following cell to see a sample of the data in CoNLL format.  As you can see, each line in the file represents a word and its labeled named entity tag in BIO format.  A blank line is used to seperate sentences.

In [4]:
!head -n 20 train

-DOCSTART-	O
	
EU	I-ORG
rejects	O
German	I-MISC
call	O
to	O
boycott	O
British	I-MISC
lamb	O
.	O
	
Peter	I-PER
Blackburn	I-PER
	
BRUSSELS	I-LOC
1996-08-22	O
	
The	O
European	I-ORG


## Reading in the Data

Below we proivide a bit of code to read in data in the CoNLL format.  This also reads in the filtered GloVe embeddings, to save you some effort - we will discuss this more later.

In [5]:
import re

#Read in the training data.
def read_conll_format(filename):
  (words, tags, currentSent, currentTags) = ([],[],['-START-'],['START'])
  for line in open(filename).readlines():
    line = line.strip()
    #print(line)
    if line == "":
      currentSent.append('-END-')
      currentTags.append('END')
      words.append(currentSent)
      tags.append(currentTags)
      (currentSent, currentTags) = (['-START-'], ['START'])
    else:
      (word, tag) = line.split()
      currentSent.append(word)
      currentTags.append(tag)
  return (words, tags)

#Read GloVe embeddings.
def read_GloVe(filename):
  embeddings = {}
  for line in open(filename).readlines():
    #print(line)
    fields = line.strip().split(" ")
    word = fields[0]
    embeddings[word] = [float(x) for x in fields[1:]]
  return embeddings

GloVe = read_GloVe("glove.840B.300d.conll_filtered.txt")

(sentences_train, tags_train) = read_conll_format("train")
(sentences_dev, tags_dev)     = read_conll_format("dev")

def sentences2char(sentences):
  return [[['start'] + [c for c in w] + ['end'] for w in l] for l in sentences]

sentencesChar = sentences2char(sentences_train)

print(sentences_train[2])
print(tags_train[2])

print(GloVe['the'])

['-START-', 'Peter', 'Blackburn', '-END-']
['START', 'I-PER', 'I-PER', 'END']
[0.27204, -0.06203, -0.1884, 0.023225, -0.018158, 0.0067192, -0.13877, 0.17708, 0.17709, 2.5882, -0.35179, -0.17312, 0.43285, -0.10708, 0.15006, -0.19982, -0.19093, 1.1871, -0.16207, -0.23538, 0.003664, -0.19156, -0.085662, 0.039199, -0.066449, -0.04209, -0.19122, 0.011679, -0.37138, 0.21886, 0.0011423, 0.4319, -0.14205, 0.38059, 0.30654, 0.020167, -0.18316, -0.0065186, -0.0080549, -0.12063, 0.027507, 0.29839, -0.22896, -0.22882, 0.14671, -0.076301, -0.1268, -0.0066651, -0.052795, 0.14258, 0.1561, 0.05551, -0.16149, 0.09629, -0.076533, -0.049971, -0.010195, -0.047641, -0.16679, -0.2394, 0.0050141, -0.049175, 0.013338, 0.41923, -0.10104, 0.015111, -0.077706, -0.13471, 0.119, 0.10802, 0.21061, -0.051904, 0.18527, 0.17856, 0.041293, -0.014385, -0.082567, -0.035483, -0.076173, -0.045367, 0.089281, 0.33672, -0.22099, -0.0067275, 0.23983, -0.23147, -0.88592, 0.091297, -0.012123, 0.013233, -0.25799, -0.02972, 0.0167

## Mapping Tokens to Indices

As in the last project, we will need to convert words in the dataset to numeric indices, so they can be presented as input to a neural network.  Code to handle this for you with sample usage is provided below.

In [6]:
#Create mappings between tokens and indices.

from collections import Counter
import random

#Will need this later to remove 50% of words that only appear once in the training data from the vocabulary (and don't have GloVe embeddings).
wordCounts = Counter([w for l in sentences_train for w in l])
charCounts = Counter([c for l in sentences_train for w in l for c in w])
singletons = set([w for (w,c) in wordCounts.items() if c == 1 and not w in GloVe.keys()])
charSingletons = set([w for (w,c) in charCounts.items() if c == 1])

#Build dictionaries to map from words, characters to indices and vice versa.
#Save first two words in the vocabulary for padding and "UNK" token.
word2i = {w:i+2 for i,w in enumerate(set([w for l in sentences_train for w in l] + list(GloVe.keys())))}
char2i = {w:i+2 for i,w in enumerate(set([c for l in sentencesChar for w in l for c in w]))}
i2word = {i:w for w,i in word2i.items()}
i2char  = {i:w for w,i in char2i.items()}

#Tag dictionaries.
tag2i = {w:i for i,w in enumerate(set([t for l in tags_train for t in l]))}
i2tag = {i:t for t,i in tag2i.items()}

#When training, randomly replace singletons with UNK tokens sometimes to simulate situation at test time.
def getDictionaryRandomUnk(w, dictionary, train=False):
  if train and (w in singletons and random.random() > 0.5):
    return 1
  else:
    return dictionary.get(w, 1)

#Map a list of sentences from words to indices.
def sentences2indices(words, dictionary, train=False):
  #1.0 => UNK
  return [[getDictionaryRandomUnk(w,dictionary, train=train) for w in l] for l in words]

#Map a list of sentences containing to indices (character indices)
def sentences2indicesChar(chars, dictionary):
  #1.0 => UNK
  return [[[dictionary.get(c,1) for c in w] for w in l] for l in chars]

#Indices
X       = sentences2indices(sentences_train, word2i, train=True)
X_char  = sentences2indicesChar(sentencesChar, char2i)
Y       = sentences2indices(tags_train, tag2i)

print(i2word[253])

#Print out some examples of what the dev inputs will look like
for i in range(10):
  print(" ".join([i2word.get(w,'UNK') for w in X[i]]))

arm
-START- -DOCSTART- -END-
-START- EU rejects German call to boycott British lamb . -END-
-START- Peter Blackburn -END-
-START- BRUSSELS 1996-08-22 -END-
-START- The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep . -END-
-START- Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . -END-
-START- " We do n't support any such recommendation because we do n't see any grounds for it , " the Commission 's chief spokesman Nikolaus van der Pas told a news briefing . -END-
-START- He said further scientific study was required and if it was found that action was needed it should be taken by the European Union . -END-
-START- He said a proposal last month by EU Farm Commissioner Franz Fischler to ban she

## Padding and Batching

In this assignment, you should train your models using minibatched SGD, rather than using a batch size of 1 as we did in the previous project.  When presenting multiple sentences to the network at the same time, we will need to pad them to be of the same length.

Below we provide some code to prepare batches of data to present to the network.  

**Side Note:** PyTorch includes utilities in [`torch.utils.data`](https://pytorch.org/docs/stable/data.html) to help with padding, batching, shuffling and some other things, but for this assignment we will do everything from scratch to help you see exactly how this works.

In [7]:
#Pad inputs to max sequence length (for batching)
def prepare_input(X_list):
  X_padded = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(l) for l in X_list], batch_first=True).type(torch.LongTensor)
  X_mask   = torch.nn.utils.rnn.pad_sequence([torch.as_tensor([1.0] * len(l)) for l in X_list], batch_first=True).type(torch.FloatTensor)
  return (X_padded, X_mask)

#Maximum word length (for character representations)
MAX_CLEN=32

def prepare_input_char(X_list):
  MAX_SLEN = max([len(l) for l in X_list])
  X_padded  = [l + [[]]*(MAX_SLEN-len(l))  for l in X_list]
  X_padded  = [[w[0:MAX_CLEN] for w in l] for l in X_padded]
  X_padded  = [[w + [1]*(MAX_CLEN-len(w)) for w in l] for l in X_padded]
  #X_mask    = [[1]*len(l) + [0]*(MAX_SLEN-len(l)) for l in X_list]
  return torch.as_tensor(X_padded).type(torch.LongTensor)

#Pad outputs using one-hot encoding
def prepare_output_onehot(Y_list, NUM_TAGS=max(tag2i.values())+1):
  Y_onehot = [torch.zeros(len(l), NUM_TAGS) for l in Y_list]
  for i in range(len(Y_list)):
   for j in range(len(Y_list[i])):
    Y_onehot[i][j,Y_list[i][j]] = 1.0
  Y_padded = torch.nn.utils.rnn.pad_sequence(Y_onehot, batch_first=True).type(torch.FloatTensor)
  return Y_padded

print("max slen:", max([len(x) for x in X_char]))  #Max sequence length in the training data is 39.

(X_padded, X_mask) = prepare_input(X)
X_padded_char = prepare_input_char(X_char)
Y_onehot = prepare_output_onehot(Y)

print("X_padded:", X_padded.shape)
print("X_padded_char:", X_padded_char.shape)
print("Y_onehot:", Y_onehot.shape)

max slen: 115
X_padded: torch.Size([14987, 115])
X_padded_char: torch.Size([14987, 115, 32])
Y_onehot: torch.Size([14987, 115, 10])


## **Your code starts here:** Basic LSTM Tagger (10 points)

OK, now you should have everything you need to get started.

Recall that your goal is to to implement the BiLSTM-CNN-CRF, as described in [(Ma and Hovy, 2016)](https://www.aclweb.org/anthology/P16-1101.pdf).  This is a relatively complex network with various components.  Below we provide starter code to break down your implementation into increasingly complex versions of the final model, starting with a Basic LSTM tagger.  This way you can be confident that each part is working correctly before incrementally increasing the complexity of your implementation.  This is generally a good approach to take when implementing complex models, since buggy PyTorch code is often partially working, but produces worse results than a correct implementation, so it's hard to know whether added complexities are helping or hurting.  Also, if you aren't able to match published results it's hard to know which component of your model has the problem (or even whether or not it is a problem in the published result!)

Fill in the functions marked as `TODO` in the code block below.  If everything is working correctly, you should be able to achieve an F1 score of 0.87 on the dev set and 0.83 on the test set (with GloVe embeddings).

In [8]:
class BasicLSTMtagger(nn.Module):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=32, DIM_HID=10):
        super(BasicLSTMtagger, self).__init__()
        NUM_TAGS = max(tag2i.values())+1

        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        #TODO: initialize parameters - embedding layer, nn.LSTM, nn.Linear and nn.LogSoftmax
        self.emb = nn.Embedding(len(word2i)+2, DIM_EMB)
        self.init_glove(GloVe)

        self.lstm = nn.LSTM(DIM_EMB, DIM_HID, num_layers = 2, batch_first = True, dropout=0.5, bidirectional=True)
        self.linear = nn.Linear(2*DIM_HID, NUM_TAGS)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, X, train=False):
        #TODO: Implement the forward computation.
        emb = self.emb(X)
        out, _ = self.lstm(emb)
        return self.softmax(self.linear(out))

    def init_glove(self, GloVe):
      #TODO: initialize word embeddings using GloVe (you can skip this part in your first version, if you want, see instructions below).
      self.emb.weight.requires_grad = False
      for i, e in enumerate(self.emb.weight):
        try:
          if i2word[i+2] in GloVe:
            self.emb.weight[i+2] = torch.Tensor(GloVe[i2word[i+2]])
        except KeyError:
          continue
      #self.emb = self.emb.from_pretrained(GloVe)

    def inference(self, sentences):
      X       = prepare_input(sentences2indices(sentences, word2i))[0].cuda()
      pred = self.forward(X).argmax(dim=2)
      return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
      Y_pred = self.inference(words)
      for i in range(len(words)):
        print("----------------------------")
        print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
        print("Predicted:\t", Y_pred[i])
        print("Gold:\t\t", tags[i])

    def write_predictions(self, sentences, outFile):
      fOut = open(outFile, 'w')
      for s in sentences:
        y = self.inference([s])[0]
        #print("\n".join(y[1:len(y)-1]))
        fOut.write("\n".join(y[1:len(y)-1]))  #Skip start and end tokens
        fOut.write("\n\n")

#The following code will initialize a model and test that your forward computation runs without errors.
lstm        = BasicLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output = lstm.forward(prepare_input(X[0:5])[0])
Y_onehot    = prepare_output_onehot(Y[0:5])

#Check the shape of the lstm_output and one-hot label tensors.
print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [9]:
#Read in the data

(sentences_dev, tags_dev)     = read_conll_format('dev')
(sentences_train, tags_train) = read_conll_format('train')
(sentences_test, tags_test)   = read_conll_format('test')

# Train your Model (10 points)

Next, implement the function below to train your basic BiLSTM tagger.  See [torch.nn.lstm](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html).  Make sure to save your predictions on the test set (`test_pred_lstm.txt`) for submission to GradeScope.

In [10]:
#Training

from random import sample
import tqdm  
import os
import subprocess
import random

def shuffle_sentences(sentences, tags):
  shuffled_sentences = []
  shuffled_tags      = []
  indices = list(range(len(sentences)))
  random.shuffle(indices)
  for i in indices:
    #print(len(sentences[i]), len(tags[i]))
    shuffled_sentences.append(sentences[i])
    shuffled_tags.append(tags[i])
  return (shuffled_sentences, shuffled_tags)

nEpochs = 10

def train_basic_lstm(sentences, tags, lstm):
  #optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
  #TODO: initialize optimizer
  optimizer = optim.SGD(lstm.parameters(), lr=0.01, momentum=0.9)
  f = nn.CrossEntropyLoss()

  batchSize = 10

  for epoch in range(nEpochs):
    totalLoss = 0.0

    (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
    for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
      lstm.zero_grad()
      #TODO: Implement gradient update.
      X, X_mask = prepare_input(sentences2indices(sentences_shuffled[batch:batch+10], word2i))
      y_onehot = prepare_output_onehot(sentences2indices(tags_shuffled[batch:batch+10], tag2i)).cuda()

      logProbs = lstm.forward(X.cuda())
      logProbs = torch.neg(logProbs * X_mask.cuda().unsqueeze(-1).expand(logProbs.size()))
      loss = torch.sum(torch.einsum("bij,bji->bij", logProbs.transpose(1,2), y_onehot))
      totalLoss += loss
      loss.backward()
      optimizer.step()

    print(f"loss on epoch {epoch} = {totalLoss}")
    lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
    print('conlleval:')
    print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

    if epoch % 10 == 0:
      s = sample(range(len(sentences_dev)), 5)
      lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

lstm = BasicLSTMtagger(DIM_HID=200, DIM_EMB=300).cuda()
train_basic_lstm(sentences_train, tags_train, lstm)


loss on epoch 0 = 26458.3203125
conlleval:
processed 51578 tokens with 5942 phrases; found: 6370 phrases; correct: 5041.
accuracy:  97.08%; precision:  79.14%; recall:  84.84%; FB1:  81.89
              LOC: precision:  83.22%; recall:  91.78%; FB1:  87.29  2026
             MISC: precision:  80.83%; recall:  80.04%; FB1:  80.44  913
              ORG: precision:  63.78%; recall:  82.85%; FB1:  72.07  1742
              PER: precision:  89.17%; recall:  81.76%; FB1:  85.30  1689

----------------------------
-START-/START/START M3/I-ORG/O 482.13/I-ORG/O 480.72/I-ORG/O 461.42/I-ORG/O -END-/END/END
Predicted:	 ['START', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'END']
Gold:		 ['START', 'O', 'O', 'O', 'O', 'END']
----------------------------
-START-/START/START Australia/I-LOC/I-LOC -/O/O Ian/I-PER/I-PER Healy/I-PER/I-PER (/O/O captain/O/O )/O/O ,/O/O Michael/I-PER/I-PER Bevan/I-PER/I-PER ,/O/O Damien/I-PER/I-PER -END-/END/END
Predicted:	 ['START', 'I-LOC', 'O', 'I-PER', 'I-PER', 'O', 'O', 'O'


loss on epoch 1 = 10080.7998046875
conlleval:
processed 51578 tokens with 5942 phrases; found: 6186 phrases; correct: 5216.
accuracy:  97.69%; precision:  84.32%; recall:  87.78%; FB1:  86.02
              LOC: precision:  91.52%; recall:  89.93%; FB1:  90.72  1805
             MISC: precision:  86.80%; recall:  80.59%; FB1:  83.58  856
              ORG: precision:  67.44%; recall:  84.49%; FB1:  75.01  1680
              PER: precision:  91.49%; recall:  91.64%; FB1:  91.56  1845




loss on epoch 2 = 6216.9375
conlleval:
processed 51578 tokens with 5942 phrases; found: 6284 phrases; correct: 5339.
accuracy:  97.78%; precision:  84.96%; recall:  89.85%; FB1:  87.34
              LOC: precision:  92.49%; recall:  93.90%; FB1:  93.19  1865
             MISC: precision:  85.34%; recall:  85.25%; FB1:  85.30  921
              ORG: precision:  67.84%; recall:  83.37%; FB1:  74.81  1648
              PER: precision:  92.43%; recall:  92.83%; FB1:  92.63  1850




loss on epoch 3 = 4574.44580078125
conlleval:
processed 51578 tokens with 5942 phrases; found: 6009 phrases; correct: 5243.
accuracy:  98.03%; precision:  87.25%; recall:  88.24%; FB1:  87.74
              LOC: precision:  93.77%; recall:  89.38%; FB1:  91.53  1751
             MISC: precision:  87.08%; recall:  84.06%; FB1:  85.54  890
              ORG: precision:  73.67%; recall:  87.84%; FB1:  80.14  1599
              PER: precision:  93.16%; recall:  89.47%; FB1:  91.28  1769




loss on epoch 4 = 3529.80029296875
conlleval:
processed 51578 tokens with 5942 phrases; found: 6134 phrases; correct: 5370.
accuracy:  98.16%; precision:  87.54%; recall:  90.37%; FB1:  88.94
              LOC: precision:  93.81%; recall:  94.12%; FB1:  93.97  1843
             MISC: precision:  86.31%; recall:  83.41%; FB1:  84.83  891
              ORG: precision:  75.98%; recall:  86.80%; FB1:  81.03  1532
              PER: precision:  91.43%; recall:  92.73%; FB1:  92.08  1868




loss on epoch 5 = 2577.072509765625
conlleval:
processed 51578 tokens with 5942 phrases; found: 5967 phrases; correct: 5310.
accuracy:  98.26%; precision:  88.99%; recall:  89.36%; FB1:  89.18
              LOC: precision:  91.68%; recall:  92.43%; FB1:  92.06  1852
             MISC: precision:  88.40%; recall:  85.14%; FB1:  86.74  888
              ORG: precision:  81.06%; recall:  84.56%; FB1:  82.77  1399
              PER: precision:  92.61%; recall:  91.91%; FB1:  92.26  1828




loss on epoch 6 = 2273.238525390625
conlleval:
processed 51578 tokens with 5942 phrases; found: 5999 phrases; correct: 5282.
accuracy:  98.18%; precision:  88.05%; recall:  88.89%; FB1:  88.47
              LOC: precision:  93.70%; recall:  89.82%; FB1:  91.72  1761
             MISC: precision:  86.92%; recall:  85.03%; FB1:  85.96  902
              ORG: precision:  78.34%; recall:  85.76%; FB1:  81.88  1468
              PER: precision:  90.90%; recall:  92.18%; FB1:  91.54  1868




loss on epoch 7 = 2011.0723876953125
conlleval:
processed 51578 tokens with 5942 phrases; found: 5958 phrases; correct: 5311.
accuracy:  98.24%; precision:  89.14%; recall:  89.38%; FB1:  89.26
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  93.63%; recall:  91.94%; FB1:  92.78  1804
             MISC: precision:  88.65%; recall:  85.57%; FB1:  87.09  890
              ORG: precision:  81.03%; recall:  84.71%; FB1:  82.83  1402
              PER: precision:  91.19%; recall:  92.13%; FB1:  91.66  1861




loss on epoch 8 = 1885.7159423828125
conlleval:
processed 51578 tokens with 5942 phrases; found: 6038 phrases; correct: 5379.
accuracy:  98.32%; precision:  89.09%; recall:  90.53%; FB1:  89.80
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  93.54%; recall:  92.98%; FB1:  93.26  1826
             MISC: precision:  86.35%; recall:  86.44%; FB1:  86.40  923
              ORG: precision:  82.24%; recall:  85.98%; FB1:  84.07  1402
              PER: precision:  91.25%; recall:  93.43%; FB1:  92.33  1886




loss on epoch 9 = 1679.218994140625
conlleval:
processed 51578 tokens with 5942 phrases; found: 6045 phrases; correct: 5368.
accuracy:  98.32%; precision:  88.80%; recall:  90.34%; FB1:  89.56
              LOC: precision:  95.07%; recall:  93.47%; FB1:  94.26  1806
             MISC: precision:  84.94%; recall:  85.03%; FB1:  84.99  923
              ORG: precision:  79.45%; recall:  86.20%; FB1:  82.69  1455
              PER: precision:  91.94%; recall:  92.89%; FB1:  92.41  1861



In [11]:
#Evaluation on test data
lstm.write_predictions(sentences_test, 'test_pred_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_lstm.txt | perl conlleval.pl -d "\t"

--2021-03-13 01:39:49--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.1’


conlleval.pl.1        0%[                    ]       0  --.-KB/s               
conlleval.pl.1      100%[===================>]  12.46K  --.-KB/s    in 0s      

2021-03-13 01:39:50 (123 MB/s) - ‘conlleval.pl.1’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5856 phrases; correct: 4787.
accuracy:  96.84%; precision:  81.75%; recall:  84.76%; FB1:  83.22
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  88.93%; recall:  88.13%; FB1:  88.53  1653
             MISC: pr

## Initialization with GloVe Embeddings (5 points)

If you haven't already, implement the `init_glove()` method in `BasicLSTMtagger` above.

Rather than initializing word embeddings randomly, it is common to use learned word embeddings (GloVe or Word2Vec), as discussed in lecture.  To make this simpler, we have already pre-filtered [GloVe](https://nlp.stanford.edu/projects/glove/) embeddings to only contain words in the vocabulary of the CoNLL NER dataset, and loaded them into a dictionary (`GloVe`) at the beginning of this notebook.



## Character Embeddings (10 points)

Now that you have your basic LSTM tagger working, the next step is to add a convolutional network that computes word embeddings from character representations of words.  See Figure 2 and Figure 3 in the [Ma and Hovy](https://www.aclweb.org/anthology/P16-1101.pdf) paper.  We have provided code in `sentences2input_tensors` to convert sentences into lists of word and character indices.  See also [nn.Conv1d](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html) and [MaxPool1d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html).

Make sure to save your predictions on the test set, for submission to GradeScope.  You should be able to achieve 90 F1 / 85 F1 on the dev/test sets.

In [12]:
#For F.max_pool1d()
import torch.nn.functional as F

#class CharLSTMtagger(nn.Module):
class CharLSTMtagger(BasicLSTMtagger):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10):
        super(CharLSTMtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID)
        NUM_TAGS = max(tag2i.values())+1

        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        #TODO: Initialize parameters.
        self.DIM_HID = DIM_HID
        self.char_emb = nn.Embedding(len(char2i)+2, DIM_CHAR_EMB)
        self.drop = nn.Dropout(0.5)
        self.conv = nn.Conv1d(DIM_CHAR_EMB, 30, 3)
        self.char_lstm = nn.LSTM(DIM_EMB + DIM_CHAR_EMB, DIM_HID, num_layers = 2, batch_first = True, dropout=0.5, bidirectional=True)

    def forward(self, X, X_char, train=False):
        #TODO: Implement the forward computation.
        word_emb = self.emb(X) #from basic LSTM
        char_emb = self.char_emb(X_char).flatten(start_dim = 0, end_dim = 1)

        batch_size = word_emb.shape[0]
        seq_len = word_emb.shape[1]

        conv = self.conv(self.drop(char_emb.transpose(1,2)))
        char_repr = F.max_pool1d(conv, kernel_size=conv.shape[2]).max(dim=2).values.reshape((batch_size, seq_len, -1))
        
        wc_emb = torch.cat((word_emb, char_repr), 2)

        out, _ = self.char_lstm(wc_emb)
        return self.softmax(self.linear(out))

    def sentences2input_tensors(self, sentences):
      (X, X_mask)   = prepare_input(sentences2indices(sentences, word2i))
      X_char        = prepare_input_char(sentences2indicesChar(sentences, char2i))
      return (X, X_mask, X_char)

    def inference(self, sentences):
      (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
      pred = self.forward(X.cuda(), X_char.cuda()).argmax(dim=2)
      return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
      Y_pred = self.inference(words)
      for i in range(len(words)):
        print("----------------------------")
        print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
        print("Predicted:\t", Y_pred[i])
        print("Gold:\t\t", tags[i])

lstm        = CharLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output = lstm.forward(prepare_input(X[0:5])[0], prepare_input_char(X_char[0:5]))
Y_onehot    = prepare_output_onehot(Y[0:5])

print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [22]:
#Training LSTM w/ character embeddings.

nEpochs = 10

def train_char_lstm(sentences, tags, lstm):
  #optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
  #TODO: initialize optimizer
  optimizer = optim.SGD(lstm.parameters(), lr=0.015, momentum=0.9)

  batchSize = 10

  for epoch in range(nEpochs):
    totalLoss = 0.0

    (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
    for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
      lstm.zero_grad()
      #TODO: Gradient update
      X, X_mask, X_char = lstm.sentences2input_tensors(sentences_shuffled[batch:batch+10])
      y_onehot = prepare_output_onehot(sentences2indices(tags_shuffled[batch:batch+10], tag2i)).cuda()
      
      logProbs = lstm.forward(X.cuda(), X_char.cuda())
      logProbs = torch.neg(logProbs * X_mask.cuda().unsqueeze(-1).expand(logProbs.size()))
      loss = torch.sum(torch.einsum("bij,bji->bij", logProbs.transpose(1,2), y_onehot))
      totalLoss += loss
      loss.backward()
      torch.nn.utils.clip_grad_norm_(lstm.parameters(), 5)
      optimizer.step()

    print(f"loss on epoch {epoch} = {totalLoss}")
    lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
    print('conlleval:')
    print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

    if epoch % 10 == 0:
      s = sample(range(len(sentences_dev)), 5)
      lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

char_lstm = CharLSTMtagger(DIM_HID=200, DIM_EMB=300).cuda()
train_char_lstm(sentences_train, tags_train, char_lstm)


loss on epoch 0 = 32856.1484375
conlleval:
processed 51578 tokens with 5942 phrases; found: 6202 phrases; correct: 5106.
accuracy:  97.62%; precision:  82.33%; recall:  85.93%; FB1:  84.09
              LOC: precision:  89.22%; recall:  90.15%; FB1:  89.68  1856
             MISC: precision:  74.13%; recall:  75.81%; FB1:  74.96  943
              ORG: precision:  69.34%; recall:  81.80%; FB1:  75.06  1582
              PER: precision:  90.83%; recall:  89.79%; FB1:  90.31  1821

----------------------------
-START-/START/START COLORADO/I-ORG/I-ORG 70/O/O 66/O/O .515/O/O 6/O/O -END-/END/END
Predicted:	 ['START', 'I-ORG', 'O', 'O', 'O', 'O', 'END']
Gold:		 ['START', 'I-ORG', 'O', 'O', 'O', 'O', 'END']
----------------------------
-START-/START/START Ray/I-PER/I-PER Lankford/I-PER/I-PER went/O/O 4-for-5/O/O with/O/O a/O/O pair/O/O of/O/O doubles/O/O for/O/O the/O/O Cardinals/I-ORG/I-ORG ,/O/O who/O/O won/O/O for/O/O just/O/O the/O/O third/O/O time/O/O in/O/O 11/O/O games/O/O ./O/O -END-


loss on epoch 1 = 13885.435546875
conlleval:
processed 51578 tokens with 5942 phrases; found: 6128 phrases; correct: 5303.
accuracy:  98.09%; precision:  86.54%; recall:  89.25%; FB1:  87.87
              LOC: precision:  93.18%; recall:  92.22%; FB1:  92.69  1818
             MISC: precision:  77.00%; recall:  82.43%; FB1:  79.62  987
              ORG: precision:  77.32%; recall:  84.41%; FB1:  80.71  1464
              PER: precision:  92.36%; recall:  93.21%; FB1:  92.79  1859




loss on epoch 2 = 9829.58203125
conlleval:
processed 51578 tokens with 5942 phrases; found: 6092 phrases; correct: 5358.
accuracy:  98.31%; precision:  87.95%; recall:  90.17%; FB1:  89.05
              LOC: precision:  91.47%; recall:  95.70%; FB1:  93.54  1922
             MISC: precision:  80.65%; recall:  80.91%; FB1:  80.78  925
              ORG: precision:  80.79%; recall:  86.58%; FB1:  83.59  1437
              PER: precision:  93.64%; recall:  91.91%; FB1:  92.77  1808




loss on epoch 3 = 7610.9833984375
conlleval:
processed 51578 tokens with 5942 phrases; found: 6130 phrases; correct: 5315.
accuracy:  98.16%; precision:  86.70%; recall:  89.45%; FB1:  88.06
              LOC: precision:  94.12%; recall:  89.82%; FB1:  91.92  1753
             MISC: precision:  76.78%; recall:  83.19%; FB1:  79.85  999
              ORG: precision:  77.29%; recall:  88.81%; FB1:  82.65  1541
              PER: precision:  92.92%; recall:  92.67%; FB1:  92.80  1837




loss on epoch 4 = 6075.865234375
conlleval:
processed 51578 tokens with 5942 phrases; found: 6159 phrases; correct: 5391.
accuracy:  98.30%; precision:  87.53%; recall:  90.73%; FB1:  89.10
              LOC: precision:  91.75%; recall:  94.99%; FB1:  93.34  1902
             MISC: precision:  76.12%; recall:  84.71%; FB1:  80.18  1026
              ORG: precision:  81.27%; recall:  87.70%; FB1:  84.36  1447
              PER: precision:  94.67%; recall:  91.69%; FB1:  93.16  1784




loss on epoch 5 = 4965.05517578125
conlleval:
processed 51578 tokens with 5942 phrases; found: 6092 phrases; correct: 5408.
accuracy:  98.44%; precision:  88.77%; recall:  91.01%; FB1:  89.88
              LOC: precision:  89.79%; recall:  96.24%; FB1:  92.91  1969
             MISC: precision:  79.75%; recall:  84.60%; FB1:  82.11  978
              ORG: precision:  87.63%; recall:  85.61%; FB1:  86.61  1310
              PER: precision:  93.30%; recall:  92.94%; FB1:  93.12  1835




loss on epoch 6 = 3786.713134765625
conlleval:
processed 51578 tokens with 5942 phrases; found: 6105 phrases; correct: 5412.
accuracy:  98.43%; precision:  88.65%; recall:  91.08%; FB1:  89.85
              LOC: precision:  91.90%; recall:  95.16%; FB1:  93.50  1902
             MISC: precision:  81.31%; recall:  85.90%; FB1:  83.54  974
              ORG: precision:  82.26%; recall:  87.84%; FB1:  84.96  1432
              PER: precision:  94.27%; recall:  91.97%; FB1:  93.10  1797




loss on epoch 7 = 2962.549560546875
conlleval:
processed 51578 tokens with 5942 phrases; found: 6037 phrases; correct: 5456.
accuracy:  98.60%; precision:  90.38%; recall:  91.82%; FB1:  91.09
              LOC: precision:  93.47%; recall:  95.86%; FB1:  94.65  1884
             MISC: precision:  85.94%; recall:  84.16%; FB1:  85.04  903
              ORG: precision:  84.84%; recall:  88.44%; FB1:  86.60  1398
              PER: precision:  93.57%; recall:  94.08%; FB1:  93.83  1852




loss on epoch 8 = 2541.24365234375
conlleval:
processed 51578 tokens with 5942 phrases; found: 6075 phrases; correct: 5476.
accuracy:  98.60%; precision:  90.14%; recall:  92.16%; FB1:  91.14
              LOC: precision:  93.49%; recall:  96.14%; FB1:  94.79  1889
             MISC: precision:  83.37%; recall:  85.36%; FB1:  84.35  944
              ORG: precision:  83.72%; recall:  89.34%; FB1:  86.44  1431
              PER: precision:  95.25%; recall:  93.65%; FB1:  94.44  1811




loss on epoch 9 = 1879.245361328125
conlleval:
processed 51578 tokens with 5942 phrases; found: 6041 phrases; correct: 5450.
accuracy:  98.55%; precision:  90.22%; recall:  91.72%; FB1:  90.96
              LOC: precision:  92.48%; recall:  95.70%; FB1:  94.06  1901
             MISC: precision:  83.65%; recall:  86.55%; FB1:  85.07  954
              ORG: precision:  86.69%; recall:  86.43%; FB1:  86.56  1337
              PER: precision:  93.83%; recall:  94.19%; FB1:  94.01  1849



In [23]:
#Evaluation on test set
char_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm.txt | perl conlleval.pl -d "\t"

--2021-03-13 02:21:52--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.6’


conlleval.pl.6        0%[                    ]       0  --.-KB/s               
conlleval.pl.6      100%[===================>]  12.46K  --.-KB/s    in 0s      

2021-03-13 02:21:52 (124 MB/s) - ‘conlleval.pl.6’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5818 phrases; correct: 4949.
accuracy:  97.40%; precision:  85.06%; recall:  87.62%; FB1:  86.32
              LOC: precision:  87.35%; recall:  91.91%; FB1:  89.57  1755
             MISC: precision:  67.49%; recall:  77.49%; FB1:  72.15  806
              ORG: 

## Conditional Random Fields (15 points)

Now we are ready to add a CRF layer to the `CharacterLSTMTagger`.  To train the model, implement `conditional_log_likelihood`, using the score (unnormalized log probability) of the gold sequence, in addition to the partition function, $Z(X)$, which is computed using the forward algorithm.  Then, you can simply use Pytorch's automatic differentiation to compute gradients by running backpropagation through the computation graph of the dynamic program (this should be very simple, so long as you are able to correctly implement the forward algorithm using a computation graph that is supported by PyTorch).  This approach to computing gradients for CRFs is discussed in Section 7.5.3 of the [Eisenstein Book](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)

You will also need to implement the Viterbi algorithm for inference during decoding.

After including CRF training and Viterbi decoding, you should be getting about 92 F1 / 88 F1 on the dev and test set, respectively.

**IMPORTANT:** Note that training will be substantially slower this time - depending on the efficiency of your implementation, it could take about 5 minutes per epoch (e.g. 50 minutes for 10 iterations).  It is recommended to start out training on a single batch of data (and testing on this same batch), so that you can quickly debug, making sure your model can memorize the labels on a single batch, and then optimize your code.  Once you are fairly confident your code is working properly, then you can train using the full dataset.  We have provided a (commented out) line of code to switch between training on a single batch and the full dataset below.

**Hint #1:** While debugging your implementation of the Forward algorithm it is helpful to look at the loss during training.  The loss should never be less than zero (the log-likelihood should always be negative).

**Hint #2:** To sum log-probabilities in a numerically stable way at the end of the Forward algorithm, you will want to use [`torch.logsumexp`](https://pytorch.org/docs/stable/generated/torch.logsumexp.html).

In [15]:
#For F.max_pool1d()
import torch.nn.functional as F

class LSTM_CRFtagger(CharLSTMtagger):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10, N_TAGS=max(tag2i.values())+1):
        super(LSTM_CRFtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID, DIM_CHAR_EMB=DIM_CHAR_EMB)

        #TODO: Initialize parameters.
        self.N_TAGS = N_TAGS
        self.transitionWeights = nn.Parameter(torch.zeros((N_TAGS, N_TAGS), requires_grad=True))
        nn.init.normal_(self.transitionWeights)

        self.transitionWeights.data[:, tag2i['START']] = -10000
        self.transitionWeights.data[tag2i["END"], :] = -10000


    def gold_score(self, lstm_scores, Y):
      #TODO: compute score of gold sequence Y (unnormalized conditional log-probability)
      score = 0
      sLen = len(Y)
      for i in range(0, sLen):
          score += lstm_scores[i, Y[i]]
      for i in range(1, sLen):
          score += self.transitionWeights[Y[i - 1], Y[i]]
      return score

    #Forward algorithm for a single sentence
    #Efficiency will eventually be important here.  We recommend you start by 
    #training on a single batch and make sure your code can memorize the 
    #training data.  Then you can go back and re-write the inner loop using 
    #tensor operations to speed things up.
    def forward_algorithm(self, lstm_scores, sLen):
      #TODO: implement forward algorithm.
      alpha = torch.Tensor(1, self.N_TAGS).cuda()
      alpha[0] = lstm_scores[0, :]

      for i in range(1, sLen):
        emissions = lstm_scores[i, :].expand(self.N_TAGS, self.N_TAGS)
        alpha_t = alpha.view(-1, 1).expand(self.N_TAGS, self.N_TAGS)
        scores = emissions + self.transitionWeights + alpha_t
        alpha = torch.logsumexp(scores, dim=0)

      return torch.logsumexp(alpha, dim=0)

    def conditional_log_likelihood(self, sentences, tags, train=True):
      #Todo: compute conditional log likelihood of Y (use forward_algorithm and gold_score)
      total_score = 0
      (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
      lstm_scores = self.forward(X.cuda(), X_char.cuda())
      for s in range(len(sentences)):
        gold_score = self.gold_score(lstm_scores[s], sentences2indices(tags, tag2i)[s])
        forward_score = self.forward_algorithm(lstm_scores[s], len(sentences[s]))
        total_score += -(gold_score - forward_score)
      return total_score

    def viterbi(self, lstm_scores, sLen):
      #TODO: Implement Viterbi algorithm, soring backpointers to recover the argmax sequence.  Returns the argmax sequence in addition to its unnormalized conditional log-likelihood.
      alpha = torch.Tensor(1, self.N_TAGS).cuda()
      alpha[0] = lstm_scores[0, :]

      backpointers = []
      for i in range(1, sLen):
        emissions = lstm_scores[i, :].expand(self.N_TAGS, self.N_TAGS)
        alpha_t = alpha.view(-1, 1).expand(self.N_TAGS, self.N_TAGS)
        scores = emissions + self.transitionWeights + alpha_t
        val, idx = torch.max(scores, dim=0)
        backpointers.append(idx)
        alpha = val

      best_idx = torch.argmax(alpha)
      score = alpha[best_idx]
      best_path = [best_idx]

      for tag in backpointers[::-1]:
        best_idx = tag[best_idx]
        best_path.append(best_idx)

      return (torch.as_tensor(best_path[::-1]), score)

    #Computes Viterbi sequences on a batch of data.
    def viterbi_batch(self, sentences):
      viterbiSeqs = []
      (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
      lstm_scores = self.forward(X.cuda(), X_char.cuda())
      for s in range(len(sentences)):
        (viterbiSeq, ll) = self.viterbi(lstm_scores[s], len(sentences[s]))
        viterbiSeqs.append(viterbiSeq)
      return viterbiSeqs

    def forward(self, X, X_char, train=False):
      #TODO: Implement the forward computation.
      word_emb = self.emb(X) #from basic LSTM
      char_emb = self.char_emb(X_char).flatten(start_dim = 0, end_dim = 1)

      batch_size = word_emb.shape[0]
      seq_len = word_emb.shape[1]

      conv = self.conv(self.drop(char_emb.transpose(1,2)))
      char_repr = F.max_pool1d(conv, kernel_size=conv.shape[2]).max(dim=2).values.reshape((batch_size, seq_len, -1))
      
      wc_emb = torch.cat((word_emb, char_repr), 2)
      
      out, _ = self.char_lstm(wc_emb)
      return self.linear(out)

    def print_predictions(self, words, tags):
      Y_pred = self.inference(words)
      for i in range(len(words)):
        print("----------------------------")
        print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
        print("Predicted:\t", [Y_pred[i][j] for j in range(len(words[i]))])
        print("Gold:\t\t", tags[i])

    #Need to use Viterbi this time.
    def inference(self, sentences, viterbi=True):
      pred = self.viterbi_batch(sentences)
      return [[i2tag[pred[i][j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]
  
lstm_crf = LSTM_CRFtagger(DIM_EMB=300).cuda()

In [16]:
print(lstm_crf.conditional_log_likelihood(sentences_dev[0:3], tags_dev[0:3]))

tensor(60.5551, device='cuda:0', grad_fn=<AddBackward0>)


In [17]:
#CharLSTM-CRF Training
import tqdm
import os
import subprocess
import random

nEpochs=10

#Get CoNLL evaluation script
os.system('wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl')

def train_crf_lstm(sentences, tags, lstm):
  #optimizer = optim.Adadelta(lstm.parameters(), lr=1.0)
  #TODO: initialize optimizer
  optimizer = optim.SGD(lstm.parameters(), lr=0.015, momentum=0.9)

  batchSize = 10

  for epoch in range(nEpochs):
    totalLoss = 0.0
    lstm.train()

    #Shuffle the sentences
    (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
    for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
      lstm.zero_grad()
      #TODO: take gradient step on a batch of data.
      loss = lstm.conditional_log_likelihood(sentences_shuffled[batch:batch+10], tags_shuffled[batch:batch+10])
      totalLoss += loss
      loss.backward()
      torch.nn.utils.clip_grad_norm_(lstm.parameters(), 5)
      optimizer.step()

    print(f"loss on epoch {epoch} = {totalLoss}")
    lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
    print('conlleval:')
    print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

    if epoch % 10 == 0:
      lstm.eval()
      s = random.sample(range(50), 5)
      lstm.print_predictions([sentences_train[i] for i in s], [tags_train[i] for i in s])   #Print predictions on train data (useful for debugging)

crf_lstm = LSTM_CRFtagger(DIM_HID=200, DIM_EMB=300, DIM_CHAR_EMB=30).cuda()
train_crf_lstm(sentences_train, tags_train, crf_lstm)             #Train on the full dataset
#train_crf_lstm(sentences_train[0:50], tags_train[0:50], crf_lstm)          #Train only the first batch (use this during development/debugging)


loss on epoch 0 = 28344.125
conlleval:
processed 51578 tokens with 5942 phrases; found: 6001 phrases; correct: 5155.
accuracy:  97.46%; precision:  85.90%; recall:  86.76%; FB1:  86.33
              LOC: precision:  93.43%; recall:  88.24%; FB1:  90.76  1735
             MISC: precision:  84.10%; recall:  76.90%; FB1:  80.34  843
              ORG: precision:  75.44%; recall:  82.25%; FB1:  78.70  1462
              PER: precision:  87.81%; recall:  93.49%; FB1:  90.56  1961

----------------------------
-START-/START/START Bonn/I-ORG/I-LOC has/O/O led/O/O efforts/O/O to/O/O protect/O/O public/O/O health/O/O after/O/O consumer/O/O confidence/O/O collapsed/O/O in/O/O March/O/O after/O/O a/O/O British/I-MISC/I-MISC report/O/O suggested/O/O humans/O/O could/O/O contract/O/O an/O/O illness/O/O similar/O/O to/O/O mad/O/O cow/O/O disease/O/O by/O/O eating/O/O contaminated/O/O beef/O/O ./O/O -END-/END/END
Predicted:	 ['START', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '


loss on epoch 1 = 11747.86328125
conlleval:
processed 51578 tokens with 5942 phrases; found: 5849 phrases; correct: 5293.
accuracy:  98.16%; precision:  90.49%; recall:  89.08%; FB1:  89.78
              LOC: precision:  88.46%; recall:  96.35%; FB1:  92.24  2001
             MISC: precision:  90.45%; recall:  80.15%; FB1:  84.99  817
              ORG: precision:  89.28%; recall:  79.49%; FB1:  84.10  1194
              PER: precision:  93.52%; recall:  93.27%; FB1:  93.39  1837




loss on epoch 2 = 8297.65625
conlleval:
processed 51578 tokens with 5942 phrases; found: 5939 phrases; correct: 5351.
accuracy:  98.21%; precision:  90.10%; recall:  90.05%; FB1:  90.08
              LOC: precision:  94.44%; recall:  94.34%; FB1:  94.39  1835
             MISC: precision:  90.44%; recall:  81.02%; FB1:  85.47  826
              ORG: precision:  85.51%; recall:  83.15%; FB1:  84.31  1304
              PER: precision:  88.96%; recall:  95.33%; FB1:  92.03  1974




loss on epoch 3 = 6359.01171875
conlleval:
processed 51578 tokens with 5942 phrases; found: 5947 phrases; correct: 5433.
accuracy:  98.41%; precision:  91.36%; recall:  91.43%; FB1:  91.40
              LOC: precision:  94.46%; recall:  94.61%; FB1:  94.53  1840
             MISC: precision:  85.11%; recall:  85.57%; FB1:  85.34  927
              ORG: precision:  88.91%; recall:  86.06%; FB1:  87.46  1298
              PER: precision:  93.09%; recall:  95.11%; FB1:  94.09  1882




loss on epoch 4 = 5133.2412109375
conlleval:
processed 51578 tokens with 5942 phrases; found: 5992 phrases; correct: 5455.
accuracy:  98.47%; precision:  91.04%; recall:  91.80%; FB1:  91.42
              LOC: precision:  94.64%; recall:  95.10%; FB1:  94.87  1846
             MISC: precision:  88.96%; recall:  84.82%; FB1:  86.84  879
              ORG: precision:  88.52%; recall:  86.28%; FB1:  87.39  1307
              PER: precision:  90.26%; recall:  96.04%; FB1:  93.06  1960




loss on epoch 5 = 4063.762939453125
conlleval:
processed 51578 tokens with 5942 phrases; found: 5968 phrases; correct: 5434.
accuracy:  98.49%; precision:  91.05%; recall:  91.45%; FB1:  91.25
              LOC: precision:  94.44%; recall:  95.32%; FB1:  94.88  1854
             MISC: precision:  91.26%; recall:  82.65%; FB1:  86.74  835
              ORG: precision:  86.99%; recall:  85.76%; FB1:  86.37  1322
              PER: precision:  90.50%; recall:  96.15%; FB1:  93.24  1957




loss on epoch 6 = 3262.5771484375
conlleval:
processed 51578 tokens with 5942 phrases; found: 5912 phrases; correct: 5428.
accuracy:  98.39%; precision:  91.81%; recall:  91.35%; FB1:  91.58
              LOC: precision:  93.52%; recall:  95.86%; FB1:  94.68  1883
             MISC: precision:  87.25%; recall:  86.12%; FB1:  86.68  910
              ORG: precision:  90.63%; recall:  82.92%; FB1:  86.60  1227
              PER: precision:  93.08%; recall:  95.60%; FB1:  94.32  1892




loss on epoch 7 = 2633.707763671875
conlleval:
processed 51578 tokens with 5942 phrases; found: 5920 phrases; correct: 5449.
accuracy:  98.52%; precision:  92.04%; recall:  91.70%; FB1:  91.87
              LOC: precision:  96.83%; recall:  93.14%; FB1:  94.95  1767
             MISC: precision:  90.16%; recall:  85.47%; FB1:  87.75  874
              ORG: precision:  85.36%; recall:  88.67%; FB1:  86.98  1393
              PER: precision:  93.37%; recall:  95.60%; FB1:  94.47  1886




loss on epoch 8 = 2128.142578125
conlleval:
processed 51578 tokens with 5942 phrases; found: 5902 phrases; correct: 5475.
accuracy:  98.64%; precision:  92.77%; recall:  92.14%; FB1:  92.45
              LOC: precision:  94.82%; recall:  95.75%; FB1:  95.29  1855
             MISC: precision:  90.31%; recall:  85.90%; FB1:  88.05  877
              ORG: precision:  91.40%; recall:  86.43%; FB1:  88.85  1268
              PER: precision:  92.80%; recall:  95.82%; FB1:  94.28  1902




loss on epoch 9 = 1603.17041015625
conlleval:
processed 51578 tokens with 5942 phrases; found: 5882 phrases; correct: 5478.
accuracy:  98.64%; precision:  93.13%; recall:  92.19%; FB1:  92.66
              LOC: precision:  95.63%; recall:  95.26%; FB1:  95.45  1830
             MISC: precision:  89.08%; recall:  85.79%; FB1:  87.40  888
              ORG: precision:  91.26%; recall:  87.99%; FB1:  89.60  1293
              PER: precision:  93.91%; recall:  95.39%; FB1:  94.64  1871



In [18]:
crf_lstm.eval()
crf_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm_crf.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm_crf.txt | perl conlleval.pl -d "\t"

--2021-03-13 02:06:55--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.4’


conlleval.pl.4        0%[                    ]       0  --.-KB/s               
conlleval.pl.4      100%[===================>]  12.46K  --.-KB/s    in 0s      

2021-03-13 02:06:55 (111 MB/s) - ‘conlleval.pl.4’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5602 phrases; correct: 5011.
accuracy:  97.75%; precision:  89.45%; recall:  88.72%; FB1:  89.08
              LOC: precision:  92.80%; recall:  91.25%; FB1:  92.02  1640
             MISC: precision:  78.23%; recall:  74.22%; FB1:  76.17  666
              ORG: 

## Gradescope

Gradescope allows you to add multiple files to your submission. Please submit this notebook along with the test set prediction:
* test_pred_lstm.txt
* test_pred_cnn_lstm.txt
* test_pred_cnn_lstm_crf.txt
* NER_release.ipynb

To download this notebook, go to `File > Download.ipynb`. You can download the predictions from Colab by clicking the folder icon on the left and finding them under Files. 

Please make sure that you name the files as specified above. You will be able to see the test set accuracy for your predictions. However, the final score will be assigned later based on accuracy and implementation. 

When submitting the .ipynb notebook, please make sure that all the cells run when executed in order starting from a fresh session. If the code doesn't take too long to run, you can re-run everything with `Runtime -> Restart and run all`

You can submit multiple times before the deadline and choose the submission which you want to be graded by going to `Submission History` on gradescope.
